# Surprisal 실습

## 초기 설정

아래 셀을 실행하여 초기 설정을 수행해 주세요.

In [ ]:
!git clone https://github.com/gyulukeyi/DeepKLM.git

Cloning into 'DeepKLM'...
remote: Enumerating objects: 136, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 136 (delta 30), reused 130 (delta 24), pack-reused 0
Receiving objects: 100% (136/136), 27.97 MiB | 14.00 MiB/s, done.
Resolving deltas: 100% (30/30), done.


In [ ]:
%cd DeepKLM

/content/DeepKLM


In [ ]:
!bash ./scripts/setup.sh

In [37]:
import torch
import sys

import pandas as pd

from sys import platform
from os import path
from torch import device
from transformers import AdamW, BertConfig, BertModel, BertTokenizer, BertForMaskedLM, AutoTokenizer, AutoModelForMaskedLM
from bertviz_lin.pytorch_pretrained_bert import BertForTokenClassification

In [6]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


## 모델 불러오기

### 영어

BERT(large, uncased)를 불러옵니다.

In [ ]:
model_name_eng = 'bert-large-uncased'
tokenizer_eng = BertTokenizer.from_pretrained(model_name_eng)
model_eng = BertForMaskedLM.from_pretrained(model_name_eng, output_attentions=True)

#model_eng = BertForTokenClassification.from_pretrained('bert-large-uncased', num_labels=2)


### 한국어
KR-BERT를 불러옵니다.

In [43]:
model_name_kr= "snunlp/KR-BERT-char16424" #huggingface 설명명 다시 한번 간단하게만만 진행
tokenizer_kr = BertTokenizer.from_pretrained(model_na me_kr, do_lower_case=False)
model_kr =BertForMaskedLM.from_pretrained(model_name_kr, output_attentions=True)

Some weights of the model checkpoint at snunlp/KR-BERT-char16424 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## 주의할 점

- 컴퓨터의 입장에서 최소대립쌍이 맞는지 확인하여야 합니다.
    - 가령, 철수는 자신을/자기를 사랑한다.에서, 
    - 자신, 자기만 바뀐 것이 아니라
    - 을/를 토큰 또한 바뀌었기 떄문에
    - 최소대립쌍이 아닙니다.
- 버트에 "등록된" 단어인지 확인하여야 합니다.
    - 효율성을 위해 버트는 '바이트 페어 인코딩'이라는 것을 수행합니다.
    - 따라서, 단어가 (형태소와는 상관 없는) 단어보다 작은 단위로 나누어서 등록되어 있을 수 있습니다.
    - 키워드가 \[UNK\]로 인식되지 않았는지 확인하셔야 합니다.

# (1차원) 요인설계실험

## 텍스트 설정

* 사용법
    * 공통되는 토큰을 [MASK]로 치환합니다.
    * 입력으로 들어가는 토큰을 키워드로 지정합니다.

In [10]:
text = """
철수가 영희[MASK] 좋아한다."""

## Surprisal

In [44]:
bert_token_surprisal(text, ["을", "를"], model_kr, tokenizer_kr, device)

Experimenting sentences:

철수가 영희[MASK] 좋아한다.
철수는 영희[MASK] 좋아한다.
Experimenting words: 을 를 

Tokenization result: 
['[CLS]', '철수', '##가', '영', '##희', '[MASK]', '좋아', '##한다', '.', '[SEP]']

Recognized Words: 을 를 
17.773056  을
13.31683  를


Tokenization result: 
['[CLS]', '철수', '##는', '영', '##희', '[MASK]', '좋아', '##한다', '.', '[SEP]']

Recognized Words: 을 를 
17.566628  을
13.3926525  를





[(' 철수가 영희 [MASK] 좋아한다. ', '을', 17.773056),
 (' 철수가 영희 [MASK] 좋아한다. ', '를', 13.31683),
 (' 철수는 영희 [MASK] 좋아한다. ', '을', 17.566628),
 (' 철수는 영희 [MASK] 좋아한다. ', '를', 13.3926525)]

# (2차원) 요인설계실험

## 텍스트 설정

* 사용법
    * 공통되는 토큰을 [MASK]로 치환합니다.
    * 입력으로 들어가는 토큰을 키워드로 지정합니다.

In [12]:
text = """
철수가 영희[MASK] 좋아한다.
철수는 영희[MASK] 좋아한다."""

## Surprisal

In [45]:
bert_token_surprisal(text, ["을", "를"], model_kr, tokenizer_kr, device)

Experimenting sentences:

철수가 영희[MASK] 좋아한다.
철수는 영희[MASK] 좋아한다.
Experimenting words: 을 를 

Tokenization result: 
['[CLS]', '철수', '##가', '영', '##희', '[MASK]', '좋아', '##한다', '.', '[SEP]']

Recognized Words: 을 를 
17.773056  을
13.31683  를


Tokenization result: 
['[CLS]', '철수', '##는', '영', '##희', '[MASK]', '좋아', '##한다', '.', '[SEP]']

Recognized Words: 을 를 
17.566628  을
13.3926525  를





[(' 철수가 영희 [MASK] 좋아한다. ', '을', 17.773056),
 (' 철수가 영희 [MASK] 좋아한다. ', '를', 13.31683),
 (' 철수는 영희 [MASK] 좋아한다. ', '을', 17.566628),
 (' 철수는 영희 [MASK] 좋아한다. ', '를', 13.3926525)]

# 여러 문장의의 Surprisal 확인하기

In [46]:
LANGUAGE = "Korean"

In [49]:
if LANGUAGE.lower() == "korean":
  model_name = model_name_kr
  tokenizer = tokenizer_kr

elif LANGUAGE.lower() == "english":
  model_name = model_name_eng
  tokenizer = tokenizer_eng

else:
  print("이번 튜토리얼에서는는 한국어와 영어어 데이터만을을 다룹니다.")

## Loading the data

In [28]:
filename = "/content/input.txt"

In [29]:
 with open(filename) as f:
        lines = f.readlines()

## Calcuating

The result will be saved as `output_name`.

In [35]:
output_name = "/content/output.txt"

In [36]:
f = open(output_name, 'w')
f.write("IDX\tSEN\tITEM1\tITEM2\n")
for i in range(1, len(lines)):
  line = lines[i].strip()
  if i % 10 == 0: print(i)
  each = line.split("\t")
  result = bert_token_surprisal(each[1].strip(), [each[2].strip(), each[3].strip()], mask_model, tokenizer, device, printing=False)
  scores = ""
  for res in result:
    scores += str(res[2])
    scores += "\t"    
  f.write(each[0] + "\t" + each[1] +  "\t" + scores.strip() + "\n")
f.close()

## Converting to xlsx

- Convert the output_file to xlsx to be read with MS Office Excel

In [38]:
df = pd.read_csv(output_name, sep='\t', index_col=0, header=0)

In [41]:
excel_name = path.splitext(output_name)[0] + '.xlsx'
df.to_excel(excel_name)